In [2]:
# Basic setup
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import json
import difflib
import pickle
import re
from collections import Counter
import seaborn as sb
import calendar

# Read in a prepared dataset
df = pd.read_csv('../data/from_20170214_to_20170425.csv')
df.index = pd.to_datetime(df.SentDate)
df['datetime'] = pd.to_datetime(df.SentDate)
df.index.tz_localize('UTC').tz_convert('US/Eastern')
df['date'] = [x.date() for x in df.index]
df['hour'] = [x.hour for x in df.index]
df = df[['From', 'Body', 'SentDate', 'Status', 'datetime', 'date', 'hour']]
df['month'] = df.datetime.dt.month.apply(lambda x: calendar.month_abbr[x])
df['day'] = df.datetime.dt.day
df['Body'] = df.Body.str.lower().replace(r"\s+", "")
df['id'] = df.From
df = df.sort_values(by = 'datetime')
df['Body_next'] = df.Body.shift(-1)

In [110]:
body_grp = df.groupby('Body')

In [134]:
def top_color(body_array):
    try:
        vc = body_array.value_counts()
        return vc.index[0]
    except IndexError:
        return np.nan

def top_value(body_array):
    try:
        vc = body_array.value_counts()
        return vc[0]
    except IndexError:
        return np.nan

sequence_color = body_grp.aggregate({'Body': len, 'Body_next': [top_color, top_value]}).sort_values(by=('Body', 'len'), ascending = False)
sequence_color.columns = ['Body_next', 'ct', 'tot_ct']
sequence_color = sequence_color.reset_index()

In [152]:
sequence_color['combo_score'] = 100*sequence_color.ct/sequence_color.tot_ct
sequence_color[sequence_color.tot_ct > 50].sort_values(by='combo_score', ascending = False).head(20)

,Body,Body_next,ct,tot_ct,combo_score
2,green,blue,166.0,823,20.170109
1,blue,red,181.0,951,19.032597
20,silver,gold,11.0,61,18.032787
5,black,white,102.0,578,17.647059
19,violet,red,13.0,74,17.567568
10,secret,secret,39.0,240,16.250000
0,red,blue,158.0,1037,15.236258
3,yellow,green,102.0,688,14.825581
4,pink,red,89.0,606,14.686469
7,orange,yellow,73.0,509,14.341847
